# **Chapter 1: Working with Text Data**


### **2.1 Tokenizing Text**

In [21]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [22]:
with open("the-verdict.txt","r", encoding="utf-8") as f :
    raw_text = f.read()

In [23]:
len(raw_text)

20479

### warm up text just for example

In [24]:
import re # python regular operation

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

result

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']

In [25]:
result = re.split(r'([,.]|\s)', text)

In [26]:
result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [27]:
# to get rid of the white spaced charcters(spaces in between)
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [28]:
# more complex example
text = "Hello, world. Is this -- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s])', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is this', '--', 'a test', '?']


### Now we use our actual data

In [29]:
result = re.split(r'([,.:;?_!()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
preprocessed = result

In [30]:
len(preprocessed)

4606

In [31]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

# **2.2 Converting the tokens into token IDs**

In [32]:
all_words = sorted(set(preprocessed))   # 'set' will help us to get rid of the duplicates in the data 
all_words[:20]

['!',
 '"',
 '"Ah',
 '"Be',
 '"Begin',
 '"By',
 '"Come',
 '"Destroyed',
 '"Don',
 '"Gisburns"',
 '"Grindles',
 '"Hang',
 '"Has',
 '"How',
 '"I',
 '"If',
 '"It',
 '"Jack',
 '"Money',
 '"Moon-dancers"']

In [33]:
vocab_size = len(all_words)
vocab_size

1158

In [59]:
# lets build the vacabulary for each token
vocab = {token : integer for integer, token in enumerate(all_words)}
# vocab
# it will enumarate all the words and assign unique integer in asscending order
# now we use this vocabulary to tokenize training data to token ids

In [35]:
# Tokenizing the same thing in raw python
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i : s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'[,.:;?_!"()\']|--|\s', text)

        preprocessed = [item.strip() for item in preprocessed if item and item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+(,.?!"()\'])', r'\1', text)
        return text

In [36]:
tokenizer = SimpleTokenizerV1(vocab)

In [37]:
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

In [38]:
ids = tokenizer.encode(text)
ids 

[95, 880, 1015, 633, 564, 776, 1154, 627, 104, 82, 881, 1136, 784, 823]

In [39]:
tokenizer.decode(ids)

'It s the last he painted you know Mrs Gisburn said with pardonable pride'

### **2.4 Adding Special context Tokens** 

In [40]:
text = "Hello, Do you like tea. is this-- a test?"

tokenizer.encode(text)

# here 'Hello' is not in the actual data so this is raising an error so to tackle this issue we will add soecial context tokens

KeyError: 'Hello'

In [41]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}


In [42]:
len(vocab.items())

1160

In [43]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1155)
('your', 1156)
('yourself', 1157)
('<|endoftext|>', 1158)
('<|unk|>', 1159)


In [44]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i : s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'[,.:;?_!"()\']|--|\s', text)

        preprocessed = [item.strip() for item in preprocessed if item and item.strip()]

        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+(,.?!"()\'])', r'\1', text)
        return text

In [45]:
tokenizer = SimpleTokenizerV2(vocab)
text = "Hello, do you like tea. is this-- a test?"

tokenizer.encode(text)


# Now it has no error as previous problem that we faced in the tokenizing the 'Hello' 

[1159, 386, 1154, 659, 1002, 615, 1026, 146, 1159]

In [46]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|> do you like tea is this a <|unk|>'

### **2.4 Byte pair encoding**  - this is actually one of the most used 

In [47]:
import tiktoken

In [48]:
tiktoken.__version__

'0.12.0'

In [49]:
tokenizer = tiktoken.get_encoding("gpt2")

In [50]:
tokenizer.encode("Hello world!")

[15496, 995, 0]

In [51]:
tokenizer.decode(tokenizer.encode("Hello world!"))

'Hello world!'

In [52]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        "of someunknownPlace.")                           # <|endoftext|> USUALLY USED TO CONCATINATE TEXT

tokenizer.encode(text, allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

### **2.5 Data Sampling With Sliding Window**
* Our task is to train in way that it predict the next wordings

In [53]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)  # here we are using TIKTOKEN to tokanise
len(enc_text)


5145

In [54]:
enc_sample =  enc_text[50:]
len(enc_sample)

5095

In [55]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f" x : {x}")
print(f" y :       {y}")

 x : [290, 4920, 2241, 287]
 y :       [4920, 2241, 287, 257]


In [56]:
# more example
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "------->", desired)

[290] -------> 4920
[290, 4920] -------> 2241
[290, 4920, 2241] -------> 287
[290, 4920, 2241, 287] -------> 257


In [57]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "------>", tokenizer.decode([desired]))

 and ------>  established
 and established ------>  himself
 and established himself ------>  in
 and established himself in ------>  a


In [58]:
import torch 
torch.__version__

'2.9.0+cpu'

In [60]:
from torch.utils.data import Dataset, dataloader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, strides):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        #Use a sliding window to chunk the book into overlapping sequences of max_length 
        for i in range(0, len(token_ids) - max_length, strides):
            input_chunk = token_ids[i:i + max_length + 1]
            target_chunk = token_ids[i+1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
            